# Libreta introductoria a PostgreSQL y Python.
## Por: Jonathan Martiñón

---
# PostgreSQL
Para instalar PostgreSQL en Windows podemos apoyarnos de:
* Sitio oficial: [PostgreSQL](https://www.postgresql.org/download/)
* *'The Coder Cave Esp'* y seguir su [Video](https://www.youtube.com/watch?v=RgP1njsQO0g&ab_channel=TheCoderCaveesp) de instalación
* Para agregar postgresql al bin [link](https://www.microfocus.com/documentation/idol/IDOL_12_0/MediaServer/Guides/html/English/Content/Getting_Started/Configure/_TRN_Set_up_PostgreSQL.htm)
* Para ejecutar un script basta con:
  - Para entrar a postgres `psql -U usuario`
  - Situarnos en la ruta donde está el script, entrar en postgres y ejecutar `\i Script.sql`
  - Nota: En mi caso era necesario entrar como Administrador, no sé si a todos s
---
# PsyCopG2

- Para su instalación revisar la [Documentación](https://pypi.org/project/psycopg2/).

- Mayor información en: [psycopg](https://www.psycopg.org/)
---

# Librerías

In [1]:
try:
    import psycopg2
except:
    # Por motivos curiosos, salió un warning que me indicó 
    # Que se debía instalar desde el binario
    !pip install psycopg2-binary
    import psycopg2

# Conexión
Para conocer el puerto, podemos ejecutar:

`-# SELECT *
-# FROM pg_settings
-# WHERE name = 'port';
` 

Por defecto es el 5423

In [2]:
# Realizamos la conexión a la base de datos.
conexion = psycopg2.connect(database="taller_db", user="user_t", password="usuario", host="localhost", port ="5432")

In [3]:
# Creamos un cursor, que nos permitirá "navegar" y hacer acciones
# En la base de datos
cursor=conexion.cursor()

# Execute
Como es fácil de intuir, será lo equivalente a la interfaz dónde indicamos la acción a realizar (INSERT, ALTER, DROP), claro, dependiendo de los permisos con los que el usuario cuente.

Si se ejecutan 2 seguidos, se sobreescribirá el valor del primero.

## INSERT

In [4]:
# Recomendado

instruccion = """ INSERT INTO materia (nombre) 
VALUES (%s) """
   
valores = ('Electricidad',)

cursor.execute(instruccion, valores)

In [5]:
# Como me acomodé
cursor.execute("INSERT INTO materia (nombre) VALUES ('Gimnasia'),('Artes'),('Biología')")

# CREATE

In [6]:
# Definimos nuestra instrucción
instruccion = """CREATE TABLE cuenta (
id INT PRIMARY KEY NOT NULL,
propietario TEXT NOT NULL,
cantidad INT NOT NULL,
fecha DATE NOT NULL
)"""

# Ejecutamos la instrucción
cursor.execute(instruccion)

## Excecutemany()
Permite ejecutar una serie de instrucciones

In [7]:
instruccion = "INSERT INTO cuenta (id, propietario, cantidad, fecha) VALUES (%s, %s, %s,%s)"

# Para demostración del default
valores = [(1, 'Javier', 12000,'2021-08-13'),
           (2, 'Helena', 23000,'2021-12-23'),
           (3, 'Hugo', 1200,'2021-12-23'),
           (4, 'Mónica', 5,'2021-01-07'),
           (5, 'Victor', 1203,'2021-12-02'),
           (6, 'Ángela', 1247,'2021-05-14'),
           (7, 'Esteban', 7543,'2021-02-22'),
           (8, 'María', 8765,'2021-09-12')
          ]

# Ejecutamos cada elemento de la lista de valores
cursor.executemany(instruccion, valores)

print("Datos subidos con éxito")

Datos subidos con éxito


Si quisiéramos sustituir ese faltante por la fecha de hoy, podremos apoyarnos de la librería `datetime` [Documentación](https://docs.python.org/es/3/library/datetime.html).

Cualquier duda de la notación `%Y-%m-%d` [Documentación](https://www.w3schools.com/python/python_datetime.asp)

In [8]:
from datetime import datetime

# Obtenemos el día actuarl
today = datetime.today()
# Podemos observar que se cuenta hasta con el tiempo
print("Información Actual:",today)

# Convertimos el formato de datetime a string
# Indicando tanto el orden como que únicamente deseamos
day_as_string = today.strftime('%Y-%m-%d')
print("Día:",day_as_string,type(day_as_string))


#------------------------------
# En una sóla instrucción
#------------------------------

# datetime.today().strftime('%Y-%m-%d')

Información Actual: 2021-05-13 22:55:34.470605
Día: 2021-05-13 <class 'str'>


# Fetch

## One
Extraemos el resultado de la consulta realizada. Se extrae un resultado (ONE), podemos verlo como el método POP de una cola. Si ejecutamos 'n' veces dejaremos vacía la cola y no existirán elementos.

Para almacenar los elementos y evitar perderlos, podemos almacenarlos en una variable o una lista (Dependerá del objetivo)

In [9]:
# Realizamos una consulta
cursor.execute("SELECT * FROM materia")

In [10]:
# Podemos ejecutar esta celda 'n' veces
print("De nuestra consulta, tenemos: ", cursor.fetchone(),"\n")

De nuestra consulta, tenemos:  (29, 'Electricidad        ') 



Del resultado anterior podemos observar que la estructura resultante es algo del estilo `(valor1, valor2)`. A este tipo de estructuras lo conocemos como **TUPLA** y podemos acceder a ella de la misma forma que accederíamos a una lista.

- tupla[0] 

  \>>> valor1
- tupla[1]

  \>>> valor2
  
La diferencia es que no podremos sobreescribir los valores
- tupla[0] = 'nuevo valor'

  \>>> Error!!

In [11]:
# Realizamos una consulta
cursor.execute("SELECT * FROM materia")

In [12]:
# Extrae el primer elemento del resultado
materia = cursor.fetchone()

In [13]:
# Mostramos el registro obtenido
print(materia)
# Mostramos el tipo de registro para comprobar que es una tupla
print(type(materia))

(29, 'Electricidad        ')
<class 'tuple'>


Dado que el segundo valor "Matemáticas", presenta espacios en blanco lo que se puede atribuir al espacio máximo dedicado a dicho campo, podemos apoyarnos de la función `strip()` [Documentación](https://www.w3schools.com/python/ref_string_strip.asp)

In [14]:
materia[1].strip()

'Electricidad'

Si no ejecutamos ninguna otra instrucción, podremos acceder y extraer el siguiente valor presente en nuestra consulta (Nuevamente llamando a `fetchone`).

In [15]:
# Extrae el primer elemento de los resultados que quedan
materia2 = cursor.fetchone()

In [16]:
# Mostramos el registro obtenido
print(materia2)
# Mostramos el tipo de registro para comprobar que es una tupla
print(type(materia2))

(30, 'Gimnasia            ')
<class 'tuple'>


In [17]:
materia2

(30, 'Gimnasia            ')

## All
De manera bastante intuitiva, podremos llegar a la conclusión de que la función `fetchall()` extrae todos los resultados de la consulta realizada. Al igual que el anterior, se trata de un `pop`, por lo que sólo servirá una vez.

Para almacenar los elementos y evitar perderlos, podemos almacenarlos en una variable o una lista (Dependerá del objetivo)

In [18]:
# Demostración de sobreescritura:

# Realizamos una consulta
cursor.execute("SELECT * FROM materia")
# Realizamos otra consulta
cursor.execute("SELECT * FROM alumno")

cursor.fetchall()

[]

In [19]:
# Si sólo hacemos una consulta: 

# Realizamos una consulta
cursor.execute("SELECT * FROM materia")

#Aplicando un primer fetchall
cursor.fetchall()

[(29, 'Electricidad        '),
 (30, 'Gimnasia            '),
 (31, 'Artes               '),
 (32, 'Biología            ')]

In [20]:
# Intentando un segundo fetchall()
cursor.fetchall()

[]

# Many
En caso de que no deseemos recuperar sólo una salida pero tampoco todas, digamos, algo así como el Top10, podemos hacer uso de `fetchmany()`

In [21]:
# Realizamos una consulta
cursor.execute("SELECT * FROM materia")

#Aplicando un primer fetchall
cursor.fetchmany(2)

[(29, 'Electricidad        '), (30, 'Gimnasia            ')]

# Commit
Si nosotros consultásemos la base de datos en este momento, podremos observar que no se han añadido valores, es por ello que se realiza un commit, de tal forma que se guarden los cambios.

In [22]:
connection.commit()

# Finalización

In [23]:
# Cerramos el cursor primero
cursor.close()
# Y Posteriormente cerramos la conexión
conexion.close()
# Un bonito mensaje final
print("La conexión se ha cerrado.\n\tGracias por tu visita (:")

La conexión se ha cerrado.
	Gracias por tu visita (:
